# CELDA1

In [1]:
! pip install -q ultralytics pandas matplotlib



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# CELDA2

In [2]:
from ultralytics import YOLO
from pathlib import Path
import torch, random, numpy as np
import pandas as pd
import matplotlib.pyplot as plt

seed=42
random.seed(seed); np.random.seed(seed); torch.manual_seed(seed); 
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

data_yaml = r"C:\Users\MSI LAPTOP\Desktop\Modelo para clasificar\Frutas.v1i.yolov8\data.yaml"
runs_dir = Path("runs/detect")


Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\MSI LAPTOP\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# CELDA3

In [3]:
model = YOLO("yolov8s.pt")
results = model.train(
    data=data_yaml,
    epochs=100,
    imgsz=640,
    batch=16,
    device=0 if torch.cuda.is_available() else "cpu",
    workers=4,
    lr0=0.01,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    optimizer="SGD",
    cos_lr=True,
    augment=True,
    pretrained=True,
    patience=20,
    degrees=15.0,
    translate=0.1,
    scale=0.15,
    shear=0.1,
    fliplr=0.5,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    mosaic=0.7,
    mixup=0.1,
    label_smoothing=0.0
)


WARNING 'label_smoothing' is deprecated and will be removed in the future.
Ultralytics 8.3.204  Python-3.11.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=C:\Users\MSI LAPTOP\Desktop\Modelo para clasificar\Frutas.v1i.yolov8\data.yaml, degrees=15.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=0.7, multi_scale=False

# CELDA4

In [2]:
from ultralytics import YOLO
from pathlib import Path

if 'data_yaml' not in locals():
    data_yaml = r"C:\Users\MSI LAPTOP\Desktop\Modelo para clasificar\Frutas.v1i.yolov8\data.yaml"

def latest_run(dirpath):
    d = Path(dirpath)
    runs = sorted([p for p in d.glob("train*") if p.is_dir()], key=lambda p: p.stat().st_mtime, reverse=True)
    return runs[0] if runs else None

base = latest_run(r"C:\Users\MSI LAPTOP\runs\detect")
if base is None:
    base = latest_run("runs/detect")

best = base.joinpath("weights/best.pt") if base else None

try:
    model
except NameError:
    model = None

if model is None:
    if best and best.exists():
        model = YOLO(str(best))
    else:
        model = YOLO("yolov8s.pt")

val_results = model.val(data=data_yaml)
print(base)
print(best, best.exists() if best else False)


Ultralytics 8.3.204  Python-3.11.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
YOLOv8s summary (fused): 72 layers, 11,156,544 parameters, 0 gradients, 28.6 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 443.1333.5 MB/s, size: 36.9 KB)
val: Scanning C:\Users\MSI LAPTOP\Desktop\Modelo para clasificar\Frutas.v1i.yolov8\valid\labels.cache... 195 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 195/195 199.7Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 13/13 5.7it/s 2.3s0.1s
                   all        195        383     0.0327     0.0305     0.0184     0.0161
                person         34         78     0.0208      0.141     0.0142     0.0109
               bicycle         47         87      0.143     0.0115     0.0776     0.0698
                   car         43         78          0          0          0          0
            motorcycle         40         64          0   

# CELDA5

In [5]:
from pathlib import Path
import matplotlib
try:
    matplotlib.use('module://matplotlib_inline.backend_inline')
except Exception:
    try:
        matplotlib.use('TkAgg')
    except Exception:
        pass
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image

bases = [
    Path(r"C:\Users\MSI LAPTOP\runs\detect"),
    Path.home() / "runs/detect",
    Path("runs/detect")
]
runs = []
for b in bases:
    if b.exists():
        runs += [p for p in b.glob("train*") if (p / "results.csv").exists()]
if not runs:
    raise FileNotFoundError("No se encontró results.csv en runs/detect.")
run = sorted(runs, key=lambda p: p.stat().st_mtime, reverse=True)[0]
df = pd.read_csv(run / "results.csv")

plt.figure()
plt.plot(df["epoch"], df["metrics/precision(B)"], label="precision")
plt.plot(df["epoch"], df["metrics/recall(B)"], label="recall")
plt.plot(df["epoch"], df["metrics/mAP50(B)"], label="mAP50")
plt.plot(df["epoch"], df["metrics/mAP50-95(B)"], label="mAP50-95")
plt.xlabel("epoch"); plt.ylabel("score"); plt.legend(); plt.grid(True); plt.tight_layout(); plt.show()

cm_path = None
candidatos = [
    run/"confusion_matrix.png",
    run/"confusion_matrix_normalized.png",
    run/"val"/"confusion_matrix.png",
    run/"val"/"confusion_matrix_normalized.png"
]
for p in candidatos:
    if p.exists():
        cm_path = p
        break

if cm_path:
    img = np.asarray(Image.open(cm_path))
    plt.figure()
    plt.imshow(img)
    plt.axis("off")
    plt.tight_layout()
    plt.show()
else:
    raise FileNotFoundError("No se encontró la matriz de confusión. Ejecute model.val(plots=True) para generarla.")


<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

# CELDA6

In [ ]:
from ultralytics import YOLO
from pathlib import Path

def latest(d, pat):
    d = Path(d)
    c = [p for p in d.glob(pat) if p.is_dir()]
    return sorted(c, key=lambda p: p.stat().st_mtime, reverse=True)[0] if c else None

candidates = [Path(r"C:\Users\MSI LAPTOP\runs\detect"), Path.home()/"runs/detect", Path("runs/detect")]
train_run = None
for base in candidates:
    if base.exists():
        r = latest(base, "train*")
        if r:
            train_run = r if train_run is None or r.stat().st_mtime > train_run.stat().st_mtime else train_run

best = train_run/"weights/best.pt" if train_run else None
model = YOLO(str(best)) if best and best.exists() else YOLO("yolov8s.pt")

export_path = model.export(format="onnx", dynamic=True, opset=12)
print(export_path)


Ultralytics 8.3.204  Python-3.11.9 torch-2.5.1+cu121 CPU (13th Gen Intel Core i7-13620H)
Model summary (fused): 72 layers, 11,127,519 parameters, 0 gradients, 28.4 GFLOPs

PyTorch: starting from 'C:\Users\MSI LAPTOP\runs\detect\train\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 9, 8400) (21.5 MB)
requirements: Ultralytics requirement ['onnxruntime'] not found, attempting AutoUpdate...
   ---------------------------------------- 0.0/13.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.4 MB ? eta -:--:--
   - -------------------------------------- 0.5/13.4 MB 2.1 MB/s eta 0:00:07
   - -------------------------------------- 0.5/13.4 MB 2.1 MB/s eta 0:00:07
   --- ------------------------------------ 1.0/13.4 MB 1.6 MB/s eta 0:00:08
   --- ------------------------------------ 1.3/13.4 MB 1.4 MB/s eta 0:00:09
   --- ------------------------------------ 1.3/13.4 MB 1.4 MB/s eta 0:00:09
   ----- ---------------------------------- 1.8/13.4

: 